# Applications of SVD (Part I)

In class, we have been discussing the *Singular Value Decomposition*. This will be the basis for all our remaining labs, as it is one of the most widely applied concepts in data science, at least as far as linear algebra is concerned. First, a quick reminder of where we are.

> ## Make a copy of this notebook (File menu -> Make a Copy...)

## Preliminaries and Definitions

**Question 1** Given a matrix $A$, what can you say about the matrix $A^TA$? What does that imply about the eigenvalues of $A^TA$?

**Question 2** Given a matrix $a$, define the following concepts:
1. The *left singular vectors* of $A$;
1. The *right singular vectors* $A$;
1. The *singular values* of $A$.

**Question 3** If $A$ is a matrix, then the SVD of $A$ is a decomposition $A=U\Sigma V^T$. By referring to Question 2, define $U$, $\Sigma$, and $V$.

**Question 4** Using previous work on eigenvectors and eigenvalues, compute the SVD of  $$A=\begin{bmatrix}2&-4\\4&4\end{bmatrix}.$$ You can do this by hand, by using code you've developed, or by using eigenvalue and eigenvector functions built into NumPy. Pay attention to the order of the eigenvalues in $\Sigma$ (you should order them in decreasing order) and make sure the columns of $U$ and $V$ correspond! Make sure you test your answer!

**Note:** If you do this by hand, you can get nice exact values! It's a good algebra exercise!

### Using Built-in NumPy Functions

The command `np.linalg.svd(A)` returns the matrices $U$, the diagonal of the matrix $\Sigma$, and the matrix $V^T$ (note that $V$ is outputted transposed!). In order to convert the diagonal of $\Sigma$ into the full matrix, use the `np.diag(D)` command that takes a diagonal and returns the corresponding matrix.

**Question 5** Using the same matrix $A$ as in the previous question, use these built in commands to compute the SVD of $A$. Test the results and comment on any differences from your answer in Question 4.

### A Quick Utility Function

We can rewrite the SVD as 

$$A=U\Sigma V^T = \sigma_1 u_1 v_1^T + \sigma_2 u_2v_2^T + \ldots + \sigma_r u_rv_r^T,$$

where $r$ is the rank of $A$, the $u_i$'s are the left singular vectors, the $v_i$'s the right singular vectors, and the $\sigma_i$'s the singular values. Note that we are taking *outer products* between the singular vectors. Since the SVD is written in decreasing order of singular values, the later $\sigma_i$'s are often very small compared to the earlier ones, and so those last terms contribute little to the matrix $A$. Therefore, for any $k<r$: 

$$A\approx \sigma_1 u_1 v_1^T + \sigma_2 u_2v_2^T + \ldots + \sigma_k u_kv_k^T.$$

In fact, this is the best rank $k$ approximation of $A$.

**Definition** if $A=U\Sigma V^T$ is the the SVD of $A$, then the *truncated SVD of rank $k$* of $A$ is $U_k\Sigma_kV^T_k$, where $U_k$ is the first $k$ columns of $u$, $\Sigma_k$ is the diagonal matrix consisting of the first $k$ entries in $\Sigma$, and $V^T_k$ is the $k$ rows of $v^T$. You can verify that this is the same as the sum of outer products above.

**Question 6** For convenience, write a function called `SVDRebuild(U,S,V,k)` that takes the SVD of an array $A$ and an integer $k$, and returns the matrix $U_k\Sigma_kV_k^T$ corresponding to the truncated SVD of rank $k$.

## Image Compression


<img src='data/puppy.bmp'>

This photo of an adorable puppy is an 8-bit grayscale image, in which each pixel can be represented by a number between 0 and 255. Zero indicates a black pixel, 255 a white pixel, and numbers in between can be thought of as a measure of brightness. In other words, we can think of this $600\times600$ pixel image as a matrix! We can load the image into a NumPy array *A* as follows:
```python
%matplotlib notebook
import matplotlib.pylab as plt

A=plt.imread('data/puppy.bmp')
```
After running this, you can use:

```python
plt.imshow(A,cmap='gray')
```
to display the image

In general, the rank of the matrix representing a natural image is its number of rows or columns (which one is it? Why?), or close to that. In this section, we will see that we can use SVD to find a lower rank matrix that gives a very similar image. We will see that this is essentially an *image compression* technique, as we need to store far less information.

**Question 7** Suppose we begin with an $m\times n$ image. Then it takes $mn$ numbers to represent it. Suppose we use a rank $k$ SVD truncation. How many numbers need to be stored? You may assume $m\geq n$. In terms of $m$ and $n$, what is the minimum value of $k$ that makes storing the compressed image take less space than the original? In our case, our image is $600\times 600$ pixels. What maximum value of $k$ could you choose for efficient storage?

### Back to the Puppy!

**Question 8** For a variety of values of $k$, examine the image you get when you rebuild the matrix from the truncated SVD of rank $k$. Take the SVD of the image, and start with small *k*. Use the following code to get an interactive widget that allows you to examine the progression:
```python
from ipywidgets import interact
import ipywidgets as widgets

fig, ax = plt.subplots()

@interact(k=widgets.IntSlider(min=1,max=300,step=1,value=1,continuous_update=True))
def update(k):
    ax.imshow(SVDRebuild(U,S,V,k),cmap='gray')
```
1. What is the smallest value of $k$ that lets you recognize that the image is a puppy?
1. What is the smallest value of $k$ that makes it hard to distinguish the compressed image from the original?

### Proportion of Information

You probably noticed in the last question that you don't really need that many columns/rows of the SVD matrices to reconstruct the original image pretty darn well. To explain this, let's look at the singular values.

**Question 9** We'll look at each singular value as a pecentage of the sum of the singular values. That will give us an idea of how much of the total information in the image is contributed by each column/row of the decomposition. 
1. Compute the sum of the singular values and store it.<br><br>
1. Divide the array $\Sigma$ by your sum and multiply by 100 (to show percentages), then print it. For ease of reading, the following command will get floats to print to four decimal places:
```python
np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})
```

(To reset to default options, call `np.set_printoptions()` with no options.)

**Question 10** We want to know how much of the information is contained in the first $k$ rows/columns.
1. Explain why we can get this by taking the *cumulative sums* of the singular values.<br><br>
1. Use the `np.cumsum(s)` command to compute the cumulative sum of your singular values. Divide the resulting array by the sum of the singular values and print your result.<br><br>
1. How many rows/columns do you need to have 50% of the original information? 60%? 70%? 80%? 90%? 95%? 
> **Note:** The command
```python
np.argmax(S>0.5)
```
will give the first index of $S$ that exceeds 0.5.
1. Plot your cumulative proportions vs. number of rows/columns. The following code gives a line plot with x-coordinates *xs* and y-coordinates *ys*:
```python
fig, ax = plt.subplots()
ax.plot(xs, ys)
```

**Question 11** 
1. Write a function `SVDthresh(S,t)` that takes the singular values $S$ of an array $A$ and a threshold $t$ between 0 and 1, and returns the number $k$ of columns/rows of the SVD needed to have $t$ of the information from the original array. <br><br>
1. Use your function together with your `SVDRebuild(U,S,V,k)` function to display the image of the puppy with various proportions of the information in the original image. Use `FloatSlider` instead of `IntSlider`.<br><br>
1. Answer the same questions as in Question 8, but with proportions of information.